In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# AGI1_05_model_training_baseline.ipynb

import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🔧 Using device: {device}")

# ✅ Load tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
model = GPT2LMHeadModel.from_pretrained('distilgpt2').to(device)

# ✅ Prepare dataset (example on alpaca_data_cleaned.csv converted to .txt)
def load_dataset(file_path, tokenizer, block_size=128):
    return TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size
    )

train_dataset = load_dataset("alpaca_data_cleaned.txt", tokenizer)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,
)

# ✅ Define training args
training_args = TrainingArguments(
    output_dir="./baseline_model_output",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=4,
    save_steps=10,
    save_total_limit=2,
    logging_steps=5
)

# ✅ Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset
)

# ✅ Start training
trainer.train()

# ✅ Save model
trainer.save_model("./baseline_model_output")
print("✅ Baseline model training complete and saved.")

2025-07-14 09:24:53.924608: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752485094.142800      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752485094.214446      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🔧 Using device: cpu


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


ValueError: Input file path alpaca_data_cleaned.txt not found

In [ ]:
# AGI1_05_model_training_baseline.ipynb

import os
import pandas as pd
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling

# ✅ Check CPU or GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🔧 Using device: {device}")

# ✅ STEP 1: Locate your input dataset
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# ✅ STEP 2: Update this path after checking the real location
csv_path = '/kaggle/input/alpaca-dataset-v1/alpaca_data_cleaned.csv'  # Update this to your actual path!

# ✅ STEP 3: Convert CSV to TXT for training
txt_file_path = '/kaggle/working/alpaca_data_cleaned.txt'

df = pd.read_csv(csv_path)

with open(txt_file_path, 'w', encoding='utf-8') as f:
    for i, row in df.iterrows():
        instruction = str(row.get('instruction', '')).strip()
        output = str(row.get('output', '')).strip()
        if instruction and output:
            line = f"### Instruction: {instruction}\n### Output: {output}\n\n"
            f.write(line)

print(f"✅ Converted CSV to TXT at: {txt_file_path}")

# ✅ STEP 4: Load tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
model = GPT2LMHeadModel.from_pretrained('distilgpt2').to(device)

# ✅ STEP 5: Prepare dataset
def load_dataset(file_path, tokenizer, block_size=128):
    return TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size
    )

train_dataset = load_dataset(txt_file_path, tokenizer)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

# ✅ STEP 6: Define training arguments
training_args = TrainingArguments(
    output_dir='/kaggle/working/baseline_model_output',
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=4,
    save_steps=10,
    save_total_limit=2,
    logging_steps=5
)

# ✅ STEP 7: Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset
)

# ✅ STEP 8: Start training
trainer.train()

# ✅ STEP 9: Save model
trainer.save_model('/kaggle/working/baseline_model_output')
print("✅ Baseline model training complete and saved to /kaggle/working/baseline_model_output")